In [2]:
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
dir = 'c:/_dados/_avito/'

In [49]:
df = pd.read_pickle(dir + 'df2.pickle')
df.fillna(0, inplace=True)
df = df.reset_index()
df['deal_chance'] = (df.deal_probability > 0.6) *1
df.head()

,index,activation_date,category_name,city,deal_probability,deal_rounded,description,image_top_1,item_id,month,...,sold_1,sold_2,sold_3,sold_4,sold_5,sold_6,sold_7,sold_8,sold_9,deal_chance
0,0,2017-03-28,Товары для детей и игрушки,Екатеринбург,0.12789,0.2,"Кокон для сна малыша,пользовались меньше месяц...",1008.0,b912c3c6a6ad,3,...,0,0,0,0,0,1,0,0,0,0
1,1,2017-03-26,Мебель и интерьер,Самара,0.00000,0.0,"Стойка для одежды, под вешалки. С бутика.",692.0,2dac0150717d,3,...,0,0,0,0,0,0,0,0,0,0
2,2,2017-03-20,Аудио и видео,Ростов-на-Дону,0.43177,0.4,"В хорошем состоянии, домашний кинотеатр с blu ...",3032.0,ba83aefab5dc,3,...,0,0,0,0,0,1,0,1,1,0
3,3,2017-03-25,Товары для детей и игрушки,Набережные Челны,0.80323,0.8,Продам кресло от0-25кг,796.0,02996f1dd2ea,3,...,1,1,1,1,1,0,0,1,1,1
4,4,2017-03-16,Автомобили,Волгоград,0.20797,0.2,Все вопросы по телефону.,2264.0,7c90be56d2ab,3,...,0,0,0,0,1,0,0,0,0,0


In [50]:
groups = df.parent_category_name.unique()
df['cat']=''
for i,group in enumerate(groups):
    df['cat_' + str(i)] = (df.parent_category_name == group) * 1
user_type = pd.get_dummies(df.user_type)
df = df.join(user_type)
df.columns

Index(['index', 'activation_date', 'category_name', 'city', 'deal_probability',
       'deal_rounded', 'description', 'image_top_1', 'item_id', 'month',
       'param_1', 'param_2', 'param_3', 'parent_category_name', 'price',
       'region', 'source', 'title', 'user_id', 'user_type', 'price_normal_log',
       'price_normal', 'price_top', 'sold_0', 'sold_1', 'sold_2', 'sold_3',
       'sold_4', 'sold_5', 'sold_6', 'sold_7', 'sold_8', 'sold_9',
       'deal_chance', 'cat', 'cat_0', 'cat_1', 'cat_2', 'cat_3', 'cat_4',
       'cat_5', 'cat_6', 'cat_7', 'cat_8', 'Company', 'Private', 'Shop'],
      dtype='object')

In [14]:
df.head()

,index,activation_date,category_name,city,deal_probability,deal_rounded,description,image_top_1,item_id,month,...,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,Company,Private,Shop
0,0,2017-03-28,Товары для детей и игрушки,Екатеринбург,0.12789,0.2,"Кокон для сна малыша,пользовались меньше месяц...",1008.0,b912c3c6a6ad,3,...,0,0,0,0,0,0,0,0,1,0
1,1,2017-03-26,Мебель и интерьер,Самара,0.00000,0.0,"Стойка для одежды, под вешалки. С бутика.",692.0,2dac0150717d,3,...,0,0,0,0,0,0,0,0,1,0
2,2,2017-03-20,Аудио и видео,Ростов-на-Дону,0.43177,0.4,"В хорошем состоянии, домашний кинотеатр с blu ...",3032.0,ba83aefab5dc,3,...,1,0,0,0,0,0,0,0,1,0
3,3,2017-03-25,Товары для детей и игрушки,Набережные Челны,0.80323,0.8,Продам кресло от0-25кг,796.0,02996f1dd2ea,3,...,0,0,0,0,0,0,0,1,0,0
4,4,2017-03-16,Автомобили,Волгоград,0.20797,0.2,Все вопросы по телефону.,2264.0,7c90be56d2ab,3,...,0,1,0,0,0,0,0,0,1,0


In [27]:
X_train = df[df['source'] == 'train']
y_train = df[df['source'] == 'train']
X_test = df[df['source'] == 'test']

predictors = X_train[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
target     = X_train['sold_0'].values
test       = X_test[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
label      = X_test['item_id'].values

model = LogisticRegression(C=5.0, random_state=0)
model.fit(predictors, target)
display(model.score(predictors,target))
y_pred = model.predict_proba(test)[:,1]

print(target.mean())
print(np.mean(y_pred))
print(y_pred)

with open(dir + 'submissions/00001.csv', 'w') as f:
    f.write('item_id,deal_probability'+ "\r\n")
    for label_, predict_ in zip(label, y_pred):
        f.write(label_ + ',' + str(np.round(predict_,4)) + "\r\n")
        
#### SCORE 0.2510

0.8608369960836065

0.13915834787791068
0.1437716476155714
[0.0794548  0.14782958 0.18697061 ... 0.08002606 0.40982857 0.07581857]


In [28]:
X_train = df[df['source'] == 'train']
y_train = df[df['source'] == 'train']
X_test = df[df['source'] == 'test']

predictors = X_train[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
target     = X_train['deal_chance'].values
test       = X_test[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
label      = X_test['item_id'].values

model = LogisticRegression(C=1.0, random_state=0)
model.fit(predictors, target)
display(model.score(predictors,target))
y_pred = model.predict_proba(test)[:,1]

print(target.mean())
print(np.mean(y_pred))
print(y_pred)

with open(dir + 'submissions/00002.csv', 'w') as f:
    f.write('item_id,deal_probability'+ "\r\n")
    for label_, predict_ in zip(label, y_pred):
        f.write(label_ + ',' + str(np.round(predict_,4)) + "\r\n")
        
#### SCORE 0.2556

0.8878353678004343

0.11216130645779235
0.11482288390438741
[0.07276252 0.13173594 0.16779363 ... 0.07365233 0.24878448 0.04735115]


In [30]:
X_train = df[df['source'] == 'train']
y_train = df[df['source'] == 'train']
X_test = df[df['source'] == 'test']

test       = X_test[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
label      = X_test['item_id'].values
predictors = X_train[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values

result = []
for g in range(10):
    target = X_train['sold_' + str(g)].values
    model  = LogisticRegression(C=1.0, random_state=0)
    model.fit(predictors, target)
    display(model.score(predictors,target))
    y_pred = model.predict_proba(test)[:,1]
    result.append(y_pred)

y_pred2 = []
for i in range(len(y_pred)):
    f = 0
    for g in range(10):
        f +=  result[g][i]/10
    y_pred2.append(f)
    
print(target.mean())
print(np.mean(y_pred))
print(y_pred)

with open(dir + 'submissions/00003.csv', 'w') as f:
    f.write('item_id,deal_probability'+ "\n")
    for label_, predict_ in zip(label, y_pred2):
        f.write(label_ + ',' + str(np.round(predict_,4)) + "\n")
        
#### SCORE 0.2511

0.8608369960836065

0.8605915563407263

0.8609114926993316

0.8607944265889064

0.8606673832531608

...

0.8610890873100336

0.8605489868460261

0.8605157294282917

0.8608137158911924

0.1391809629219701
0.14382483593094558
[0.07939672 0.14674548 0.18713147 ... 0.07996054 0.41175167 0.07421595]


In [37]:
X_train = df[df['source'] == 'train']
y_train = df[df['source'] == 'train']
X_test = df[df['source'] == 'test']

predictors = X_train[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
target     = X_train['deal_probability'].values
test       = X_test[['price_normal','price_normal_log','price_top','Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']].values
label      = X_test['item_id'].values

model = LinearRegression()
model.fit(predictors, target)
display(model.score(predictors,target))
y_pred = model.predict(test)

print(target.mean())
print(np.mean(y_pred))
print(y_pred)

with open(dir + 'submissions/00004.csv', 'w') as f:
    f.write('item_id,deal_probability\n')
    for label_, predict_ in zip(label, y_pred):
        f.write(label_ + ',' + str(np.round(predict_,4).clip(0,0.99))+ '\n')
        
#### SCORE 0.2513

0.11143490760183972

0.13913057577902177
0.13991851371100106
[0.08270784 0.15046021 0.1871146  ... 0.08344773 0.40809342 0.09394676]


In [52]:
df['price_pos'] = df['price_normal'] + 2

X_train = df[df['source'] == 'train']
y_train = df[df['source'] == 'train']
X_test = df[df['source'] == 'test']


fields = ['price_pos', 'Private','Shop','cat_0','cat_1','cat_2','cat_3','cat_4','cat_5','cat_6','cat_7']

predictors = X_train[fields].values
target     = X_train['sold_0'].values
test       = X_test[fields].values
label      = X_test['item_id'].values

model = MultinomialNB(alpha=0.1, fit_prior=True)
model.fit(predictors, target)
display(model.score(predictors,target))
y_pred = model.predict_proba(test)[:,1]

print(target.mean())
print(np.mean(y_pred))
print(y_pred)

with open(dir + 'submissions/00005.csv', 'w') as f:
    f.write('item_id,deal_probability\n')
    for label_, predict_ in zip(label, y_pred):
        f.write(label_ + ',' + str(np.round(predict_,4).clip(0,0.99))+ '\n')
        
#### SCORE 0.2526

0.8608416521220893

0.13915834787791068
0.14274093478445804
[0.08000439 0.1310949  0.18390893 ... 0.08001409 0.41585119 0.08098078]
